In [ ]:
!pip install checklist

In [ ]:
!jupyter nbextension install --py --user checklist.viewer

In [ ]:
!jupyter nbextension enable --py --user checklist.viewer

In [ ]:
!git clone https://github.com/marcotcr/checklist.git

In [ ]:
%cd checklist

In [ ]:
!pwd

In [ ]:
!pip install -e .

In [ ]:
!pip install torch

In [ ]:
!python -m spacy download en_core_web_sm

In [10]:
import pandas as pd
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
!pip install simpletransformers

In [ ]:
from simpletransformers.classification import ClassificationModel

BERT = ClassificationModel("bert", "bert-base-cased")
train_file = pd.read_csv("/content/olid-train.csv", usecols=["text", "labels"])
BERT.train_model(train_df=train_file)

In [13]:
test_path = "/content/olid-subset-diagnostic-tests.csv"
test_file = pd.read_csv(test_path)
test_file = test_file.drop(["id","labels"], axis=1)
test_file = test_file["text"].tolist()

predictions, raw_outputs = BERT.predict(to_predict=test_file)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [14]:
test_original = pd.read_csv("/content/olid-subset-diagnostic-tests.csv")
test_original_gold = test_original["labels"].tolist()
test_file = test_original["text"].tolist()

predictions, raw_outputs = BERT.predict(to_predict=test_file)

macro = precision_recall_fscore_support(test_original_gold, predictions, average="macro")

print("\n\033[1mResults for macro-average:\n--------------------------------\033[0m")
print("Precision:", macro[0], "\nRecall:", macro[1], "\nF1:", macro[2])
test_original["original prediction"] = predictions
test_original["eval 0"] = test_original["labels"] == test_original["original prediction"]



  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]


Results for macro-average:
--------------------------------
Precision: 0.8255208333333333 
Recall: 0.8 
F1: 0.7960016319869441


In [ ]:
pert_df = pd.read_csv("/content/olid-subset-diagnostic-tests.csv")
pert_gold = pert_df["labels"].tolist()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

np.random.seed(42)

percentage_typos = [0.10, 0.20, 0.30, 0.40, 0.50]

print(f"Identified Corretly Original Dataset: {len(test_original.loc[test_original['eval 0'] == True])}")

for idx, value in enumerate(percentage_typos):
  pert_df["text"] = pert_df["text"].apply(lambda x: Perturb.add_typos(string=x, typos=int(len(x)*value)))
  pert_test_file = pert_df["text"].tolist()
  predictions, raw_outputs = BERT.predict(to_predict=pert_test_file)

  macro = precision_recall_fscore_support(pert_gold, predictions, average="macro")
  print(f"\n\033[1mResults for macro-average {value}:\n--------------------------------\033[0m")
  print("Precision:", macro[0], "\nRecall:", macro[1], "\nF1:", macro[2])
  
  test_original[f"prediction {idx+1}"] = predictions
  test_original[f"eval {idx+1}"] = test_original["original prediction"] == test_original[f"prediction {idx+1}"]
  test_original[f"text {idx+1}"] = pert_df["text"]

  print(f"Identified Corretly eval {idx+1}: {len(test_original.loc[test_original[f'eval {idx+1}'] == True])}")
  print(test_original.loc[test_original[f'eval {idx+1}'] == False, f"text {idx+1}"].values[0:4])

print(test_original)




6 Negation



*   Check the add_negation function in checklist pertub.py. What kind of negations does it produces? (1 Point)



Negations:
*   is not
*   was not
*   were not
*   am not
*   are not
*   can't / couldn't
*   don't / didn't
*   won't / wouldn't
*   haven't / hadn't
*   will not



●	Look at the created negated sentences, are they linguistically correct? Provide 2–5 examples of linguistically incorrect sentences. (1 point)

In [31]:
import spacy
nlp = spacy.load("en_core_web_sm")
data = pd.read_csv("/content/olid-subset-diagnostic-tests.csv")

data["doc text"] = data["text"].apply(lambda x: nlp(x))
data["negated text"] = data["doc text"].apply(lambda x: Perturb.add_negation(doc=x))
data["labels 2"], raw_outputs = BERT.predict(to_predict=data["negated text"].tolist())

pd.set_option('display.width', 500)
pd.options.display.max_colwidth = 500
pd.set_option('display.max_columns', 500)

data["Comparison"] = data["labels"] == data["labels 2"]
print(data.loc[0:11])

# print(data.loc[data["Comparison"] == False, ["text", "negated text"]])
# print(f"Example 1: {data['negated text'].loc[0]}")
# print(f"Example 2: {data['negated text'].loc[38]}")
# print(f"Example 3: {data['negated text'].loc[95]}")



  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

       id                                                                                                                                                                                                                                                                                          text  labels  \
0   89200                                                                                                                                                                                                                                                 @USER @USER Who the hell does he think he is?       1   
1   71294                                                                                                                                                                                     #BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL       1   
2   55633                    #OrrinHatch I can’t believe this sexist , clueless

●	Check the first 10 negated messages. For which of these negated messages should the label be flipped, in your opinion? (1 point)

All First 10 Sentences are originally labelled as Offensive.
Should they change based on the added negations?

1. Should not change, sentence becomes linguistically incorrect but does not change the essence of the sentence. There is still offensive intent
2. Should not change. Offensive intent is argumented through the use and combination of words like Molotov cocktails / killed
3. Should not change, due to the use of words like sexist / clueless /old fart etc.
4. Should not change, because of the word "fucking"
5. Should not change, because of the subsentence: "I hope he rots in hell"
6. Should not change, stating the assumption that a religious group supports school shootings
7. Should not change, using words like despicable behaviour and disingenuous
8. Should not change, using the word nazi's and the word fucking
9. Should not change, dirty language
10. Should not change, offensive hashtags

In [35]:
pd.set_option('display.width', 500)
pd.options.display.max_colwidth = 500
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

print(data.loc[data["Comparison"] == True, ["text", "negated text"]])

                                                                                                                                                                                                                                                                                             text  \
0                                                                                                                                                                                                                                                   @USER @USER Who the hell does he think he is?   
2                      #OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!   
4                                                                                                                        

Correct Examples:
1. #FemiNazis are not Liberals so #MeToo only counts when they blame a non-Liberal man. #Bizarro  #KAG2018 ❤️🇺🇸 URL  

2. #StayOnYourPath #HealthQuote If you are wishing to loose fat, don't get back your wellbeing and slow down or reverse the aging process then check this super powerful natural method URL URL  


Wrong Examples:

1. @USER She Is not A ChiCom Spy..... #ClubGITMO  

2. $1500 for a phone. You all are not fucking dumb.  
